In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
%cd /lustre/scratch126/cellgen/team205/aa34/Arian/Dis2P/dis2p_reproducibility/figure_notebooks/saved_resutls

/lustre/scratch126/cellgen/team205/aa34/Arian/Dis2P/dis2p_reproducibility/figure_notebooks/saved_resutls


In [3]:
!ls

split_1_1212_delta_pearson.csv	split_2_5010_pearson.csv
split_1_1212_pearson.csv	split_4_1212_delta_pearson.csv
split_2_5010_delta_pearson.csv	split_4_1212_pearson.csv


In [25]:
def get_csv(path):
    data = pd.read_csv(path)
    data = data.rename(columns={'Unnamed: 0': 'n_degs'})
    data.index = data['n_degs']
    data = data.drop(columns=['n_degs'])
    data = data.T
    return data

In [30]:
dfs = {}
for path in os.listdir('.'):
    if '1212' in path:
        dfs[path.split('.csv')[0]] = get_csv(path)

In [313]:
def preproc_df(df):
    arrays = [
        ['Mean Pearson'] * len(df.columns) + ['Variance Pearson'] * len(df.columns),
        list(df.columns) * 2
    ]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=["Metric", "Gene Subset"])

    df = df.rename(index={'Prediction': 'Dis2p (Ours)', 'Prediction_var': 'Dis2p (Ours)_var'})
    # Reindex the columns to fit the new MultiIndex structure
    # df.columns = index
    split = int(len(df)/2)
    # Splitting the DataFrame to separate means and vars, then concatenating
    mean_df = df.iloc[:split].set_index([[row.split('_')[0] for row in df.index[:split]]], drop=True)
    var_df = df.iloc[split:].set_index([[row.split('_')[0] for row in df.index[split:]]], drop=True)
    # Concatenating mean and var DataFrames
    final_df = pd.concat([mean_df, var_df], keys=['Mean Pearson', 'Variance Pearson'], axis=1)

    # Displaying the DataFrame
    # print(final_df)
    return final_df
    
def beautify(df, name):
    def highlight_max(data):
        '''
        highlight the maximum in a Series yellow.
        '''
        attr = 'font-weight: bold'
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    styled_df = df.style.apply(highlight_max, axis=0)

    # Draw a border between Mean Pearson and Variance Pearson
    mean_cols = df.columns.get_level_values(0) == 'Mean Pearson'
    var_cols = df.columns.get_level_values(0) == 'Variance Pearson'

    border_index = df.columns.get_level_values(0).tolist().index('Variance Pearson')
    column_border_styles = {('Variance Pearson', col): 'border-left: 2px solid black;' for col in df['Variance Pearson'].columns}
    caption = name.split('_')
    if len(caption) == 5:
        caption = f"{caption[1]} (Delta Pearson)"
    else:
        caption = f"{caption[1]} (Regular Pearson)"
    styled_df = styled_df.set_caption(f"Scenario: {caption}")
    styled_df = styled_df.set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}] +
        [{'selector': f'th.col{border_index}', 'props': [('border-left', '2px solid black')]},
         {'selector': f'td.col{border_index}', 'props': [('border-left', '2px solid black')]},
         {'selector': 'th.col_heading.level0', 'props': [('border', '2px black solid')]},
         {'selector': 'th.col_heading.level1', 'props': [('border-bottom', '1px black solid')]},
         {'selector': '', 'props': [('border', '2px black solid')]},
         {'selector': 'td.col0', 'props': [('border-left', '2px black solid')]},
         {'selector': 'th.col0', 'props': [('border-left', '2px black solid')]},

         {'selector': 'caption', 'props': [('font-size', '20px')]},
        ])
    return styled_df

In [314]:
nice_dfs = {}
for name, df in dfs.items():
    print(name)
    final_df = preproc_df(df)
    styled_df = beautify(final_df, name)
    nice_dfs[name] = styled_df

split_4_1212_delta_pearson
split_2_1212_pearson
split_2_1212_delta_pearson
split_1_1212_delta_pearson
split_4_1212_pearson
split_1_1212_pearson


In [315]:
nice_dfs

{'split_4_1212_delta_pearson': <pandas.io.formats.style.Styler at 0x1522c8103b80>,
 'split_2_1212_pearson': <pandas.io.formats.style.Styler at 0x1522c8c00ee0>,
 'split_2_1212_delta_pearson': <pandas.io.formats.style.Styler at 0x1522c81469a0>,
 'split_1_1212_delta_pearson': <pandas.io.formats.style.Styler at 0x1522c810a100>,
 'split_4_1212_pearson': <pandas.io.formats.style.Styler at 0x1522c810a460>,
 'split_1_1212_pearson': <pandas.io.formats.style.Styler at 0x1522c8981160>}

In [327]:
print(nice_dfs['split_1_1212_pearson'].to_latex())
nice_dfs['split_1_1212_pearson']

\begin{table}
\caption{Scenario: 1 (Regular Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 5 & 10 & 20 & 30 & 40 & 50 & 59 & all & 5 & 10 & 20 & 30 & 40 & 50 & 59 & all \\
Dis2p (Ours) & 0.919533 & \font-weightbold 0.808732 & \font-weightbold 0.807743 & \font-weightbold 0.538114 & 0.424560 & 0.304618 & 0.241131 & 0.645892 & 0.602976 & 0.569098 & 0.546108 & 0.479476 & 0.446048 & 0.375258 & 0.346426 & \font-weightbold 0.816173 \\
Biolord & \font-weightbold 0.937612 & 0.219765 & 0.336275 & 0.483282 & 0.385091 & 0.281464 & 0.243012 & 0.738346 & \font-weightbold 0.994287 & 0.346100 & 0.405715 & 0.395603 & 0.231278 & 0.135182 & 0.108987 & 0.657048 \\
scdisinfact & 0.706087 & 0.702502 & 0.570141 & 0.5149

In [328]:
print(nice_dfs['split_1_1212_delta_pearson'].to_latex())
nice_dfs['split_1_1212_delta_pearson']

\begin{table}
\caption{Scenario: 1 (Delta Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 5 & 10 & 20 & 30 & 40 & 50 & 59 & all & 5 & 10 & 20 & 30 & 40 & 50 & 59 & all \\
Dis2p (Ours) & \font-weightbold 0.895112 & \font-weightbold 0.971713 & \font-weightbold 0.950626 & 0.780276 & 0.539566 & 0.439258 & 0.322372 & 0.095422 & \font-weightbold 0.985390 & 0.464355 & 0.408912 & \font-weightbold 0.552110 & \font-weightbold 0.588865 & \font-weightbold 0.583717 & \font-weightbold 0.587764 & \font-weightbold 0.174703 \\
Biolord & 0.891373 & 0.957008 & 0.930307 & 0.803843 & 0.348231 & 0.256173 & 0.198968 & \font-weightbold 0.251696 & 0.821761 & \font-weightbold 0.737716 & \font-weightbold 0.688731 & 0.504337 

In [329]:
print(nice_dfs['split_2_1212_pearson'].to_latex())
nice_dfs['split_2_1212_pearson']

\begin{table}
\caption{Scenario: 2 (Regular Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all \\
Dis2p (Ours) & \font-weightbold 0.959084 & \font-weightbold 0.925856 & \font-weightbold 0.869005 & \font-weightbold 0.843693 & \font-weightbold 0.584346 & \font-weightbold 0.491003 & \font-weightbold 0.373192 & 0.508499 & \font-weightbold 0.935465 & \font-weightbold 0.817717 & \font-weightbold 0.764537 & \font-weightbold 0.759832 & \font-weightbold 0.546412 & \font-weightbold 0.493312 & \font-weightbold 0.377313 & \font-weightbold 0.624210 \\
Biolord & 0.230462 & 0.411268 & 0.416182 & 0.424709 & 0.341523 & 0.336665 & 0.2

In [330]:
print(nice_dfs['split_2_1212_delta_pearson'].to_latex())
nice_dfs['split_2_1212_delta_pearson']

\begin{table}
\caption{Scenario: 2 (Delta Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all \\
Dis2p (Ours) & \font-weightbold 0.783905 & \font-weightbold 0.753967 & \font-weightbold 0.697841 & \font-weightbold 0.699264 & \font-weightbold 0.694094 & \font-weightbold 0.644820 & \font-weightbold 0.561860 & 0.402004 & \font-weightbold 0.674747 & \font-weightbold 0.649228 & \font-weightbold 0.557708 & \font-weightbold 0.557796 & 0.526000 & 0.497222 & 0.454481 & \font-weightbold 0.141953 \\
Biolord & 0.640997 & 0.641451 & 0.560167 & 0.571966 & 0.575855 & 0.551018 & 0.494881 & \font-weightbold 0.514156 & 0.637193 & 0.6354

In [331]:
print(nice_dfs['split_4_1212_pearson'].to_latex())
nice_dfs['split_4_1212_pearson']

\begin{table}
\caption{Scenario: 4 (Regular Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all \\
Dis2p (Ours) & -0.126433 & \font-weightbold 0.196108 & \font-weightbold 0.290307 & \font-weightbold 0.188150 & \font-weightbold 0.424391 & \font-weightbold 0.472288 & \font-weightbold 0.523214 & \font-weightbold 0.724956 & -0.213883 & -0.043957 & 0.028725 & 0.007048 & \font-weightbold 0.180121 & \font-weightbold 0.189966 & \font-weightbold 0.305356 & 0.663174 \\
Biolord & \font-weightbold -0.100184 & 0.171921 & 0.242283 & 0.119071 & 0.350603 & 0.392264 & 0.454844 & 0.692520 & \font-weightbold -0.110269 & \font-weightbold

In [332]:
print(nice_dfs['split_4_1212_delta_pearson'].to_latex())
nice_dfs['split_4_1212_delta_pearson']

\begin{table}
\caption{Scenario: 4 (Delta Pearson)}
\thcenter
\tdcenter
\th.col82px solid black
\td.col82px solid black
\th.col_heading.level02px black solid
\th.col_heading.level11px black solid
\2px black solid
\td.col02px black solid
\th.col02px black solid
\begin{tabular}{lrrrrrrrrrrrrrrrr}
 & \multicolumn{8}{r}{Mean Pearson} & \multicolumn{8}{r}{Variance Pearson} \\
n_degs & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all & 10 & 20 & 40 & 50 & 100 & 150 & 200 & all \\
Dis2p (Ours) & 0.247337 & 0.304460 & \font-weightbold 0.352760 & \font-weightbold 0.311429 & \font-weightbold 0.458913 & \font-weightbold 0.604161 & \font-weightbold 0.611124 & \font-weightbold 0.504747 & 0.169714 & 0.095818 & 0.076060 & 0.145672 & 0.111895 & 0.220036 & \font-weightbold 0.193079 & \font-weightbold -0.071152 \\
Biolord & \font-weightbold 0.313444 & \font-weightbold 0.330918 & 0.347008 & 0.243015 & 0.404857 & 0.539864 & 0.544130 & 0.431561 & \font-weightbold 0.180627 & \font-weightbold 0.134244 & \font-weigh